In [ ]:
import json
import numpy as np
import tensorflow as tf
from tensorflow.keras.layers import Dense, LSTM, Embedding
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.models import Sequential

# Load data
with open('intents.json') as file:
    data = json.load(file)

# Prepare data
patterns = []
tags = []
for intent in data['intents']:
    for pattern in intent['patterns']:
        patterns.append(pattern)
        tags.append(intent['tag'])

# Encode tags
encoder = LabelEncoder()
encoded_tags = encoder.fit_transform(tags)


In [ ]:
# Tokenize text data
tokenizer = Tokenizer(num_words=2000, oov_token="<OOV>")
tokenizer.fit_on_texts(patterns)
sequences = tokenizer.texts_to_sequences(patterns)
padded_sequences = pad_sequences(sequences, padding='post')

# Vocabulary size
vocab_size = len(tokenizer.word_index) + 1
input_length = padded_sequences.shape[1]


In [ ]:
model = Sequential([
    Embedding(vocab_size, 16, input_length=input_length),
    LSTM(32),
    Dense(32, activation='relu'),
    Dense(len(set(tags)), activation='softmax')
])

model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


In [ ]:
# Train the model
model.fit(padded_sequences, np.array(encoded_tags), epochs=200)

Epoch 1/200
13/13 ━━━━━━━━━━━━━━━━━━━━ 2s 7ms/step - accuracy: 0.0676 - loss: 3.6350
Epoch 2/200
13/13 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - accuracy: 0.0779 - loss: 3.6205
Epoch 3/200
13/13 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - accuracy: 0.0836 - loss: 3.5923
Epoch 4/200
13/13 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - accuracy: 0.0630 - loss: 3.5525
Epoch 5/200
13/13 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - accuracy: 0.0712 - loss: 3.5080
Epoch 6/200
13/13 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - accuracy: 0.0539 - loss: 3.4653
Epoch 7/200
13/13 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.0632 - loss: 3.3532
Epoch 8/200
13/13 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - accuracy: 0.0490 - loss: 3.1934
Epoch 9/200
13/13 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.0977 - loss: 3.0761
Epoch 10/200
13/13 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - accuracy: 0.0824 - loss: 3.0476
Epoch 11/200
13/13 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - accuracy: 0.1260 - loss: 2.9730
Epoch 12/200
13/13 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - accuracy:

In [ ]:
def chatbot_response(text):
    sequence = tokenizer.texts_to_sequences([text])
    padded = pad_sequences(sequence, maxlen=input_length, padding='post')
    prediction = model.predict(padded)
    tag = encoder.inverse_transform([np.argmax(prediction)])

    for intent in data['intents']:
        if intent['tag'] == tag:
            return np.random.choice(intent['responses'])


In [ ]:

print("Start chatting with the bot (type 'quit' to stop)")
while True:
    inp = input("You: ")
    if inp.lower() == "quit":
        break
    response = chatbot_response(inp)
    print("Bot:", response)

Start chatting with the bot (type 'quit' to stop)
You: hi
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 174ms/step
Bot: Hi there, how can I help?
You: who are you
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step
Bot: You can call me Virtus.
You: youu?
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step
Bot: Good to see you again!
You: you?
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step
Bot: I am your helper, Virtus
You: ok
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step
Bot: welcome, anything else i can assist you with?
You: what can you do?
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step
Bot: You can ask me questions regarding college, and i will try to answer them
You: what are timings of college
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step
Bot: Our university's Engineering department provides fully AC Lab with internet connection, smart classroom, Auditorium, library,canteen
You: branches of college?
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step
Bot: Our university offers Information Technology, computer Engineering, Mechanical engineering,Chemical engineering, Civil 

In [ ]:
!pip install streamlit
!pip install pyngrok

In [ ]:
# Streamlit UI
import streamlit as st
st.title("Chatbot")
st.write("Ask something about college!")

user_input = st.text_input("You: ", "")
if user_input:
    response = chatbot_response(user_input)
    st.write("Bot:", response)

2024-11-11 12:21:10.402 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-11-11 12:21:10.405 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-11-11 12:21:10.407 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-11-11 12:21:10.408 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-11-11 12:21:10.410 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-11-11 12:21:10.412 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-11-11 12:21:10.413 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-11-11 12:21:10.414 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bar

In [ ]:
st run streamlit_chatbot.py